This notebook is based on a notebook from [lesson](https://www.coursera.org/learn/advanced-computer-vision-with-tensorflow/ungradedLab/Wlfvj/implement-a-fully-convolutional-neural-network) in course "Advanced Computer Vision with TensorFlow" by DeepLearning.AI

In [2]:
import os
import zipfile
import PIL.Image, PIL.ImageFont, PIL.ImageDraw
import numpy as np

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf
from matplotlib import pyplot as plt
import tensorflow_datasets as tfds
import seaborn as sns

print("Tensorflow version " + tf.__version__)

Tensorflow version 2.5.0


In [3]:
# class_names = ['background', 'building', 'woodland', 'water', 'road']

## Load and prepare dataset

Dataset classes names and corresponding values for pixels in masks are shown below.

| Value  | Class Name    |
| -------| -------------| 
| 0      | background |
| 1      | building      |
| 2      | woodland      |
| 3      | water |
| 4      | road     |

Each mask has shape `(512, 512, 3)` where third channel is responsible for providing information about class to which particular pixel belongs. 

For example if we'll acces pixel of road **`mask_as_array[height, width, :]`** the result will be list **`[4, 4, 4]`** denoting pixel class.

For further explanation open [notebook](https://github.com/MortenTabaka/roads-semantic-segmentation-in-LandCover.ai-dataset_graduate-thesis/blob/main/notebooks/exploratory/0.1-Marcin-verify_mask_convention_for_classes.ipynb).

For this project label mask will be reshaped from `(height, width, 3)` to `(height, width, num_of_classes)` with each slice along the third axis having `1` if it belongs to the class corresponding to that slice's index else `0`.

So if a pixel is part of a road, then **`mask_as_array[height, width, :] = [0, 0, 0, 0, 1]`**

The project is aiming to distinguish roads and everything else, then there will be only two classes: **background** and **road**.


In [4]:
class_names = ['background', 'road']

In [ ]:
def map_filename_to_image_and_mask(t_filename, a_filename):
    '''
    Preprocesses the dataset by:
        * resizing the input image and label maps
        * normalizing the input image pixels
        * reshaping the label maps from (height, width, 3) to (height, width, 2)

    Args:
        t_filename (string) -- path to the raw input image
        a_filename (string) -- path to the raw annotation (label map) file

    Returns:
        image (tensor) -- preprocessed image
        annotation (tensor) -- preprocessed annotation
    '''
    # Convert image and mask files to tensors 
    img_raw = tf.io.read_file(t_filename)
    anno_raw = tf.io.read_file(a_filename)
    image = tf.io.decode_jpeg(img_raw)
    annotation = tf.io.decode_png(anno_raw)
    
    # Resize image and segmentation mask
    image = tf.image.resize(image, (height, width,))
    annotation = tf.image.resize(annotation, (height, width,))
    image = tf.reshape(image, (height, width, 3,))
    annotation = tf.cast(annotation, dtype=tf.int32)
    annotation = tf.reshape(annotation, (height, width, 1,))
    stack_list = []

    # Reshape segmentation masks
    for c in range(len(class_names)):
        mask = tf.equal(annotation[:,:,0], tf.constant(c))
        stack_list.append(tf.cast(mask, dtype=tf.int32))

    annotation = tf.stack(stack_list, axis=2)

    # Normalize pixels in the input image
    image = image/127.5
    image -= 1

    return image, annotation